In [1]:
import sys
print(sys.version)

3.12.4 (tags/v3.12.4:8e8a4ba, Jun  6 2024, 19:30:16) [MSC v.1940 64 bit (AMD64)]


In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pickle

In [14]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [17]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

print("hello")

hello


In [19]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
number_of_classes = 3
dataset_size = 100

DATA_DIR = './data'
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for j in range(number_of_classes):
        if not os.path.exists(os.path.join(DATA_DIR, str(j))):
            os.makedirs(os.path.join(DATA_DIR, str(j)))
    
        print('Collecting data for class {}'.format(j))
    
        # Read feed
        # ret, frame = cap.read()
    
        # Make detections
        # image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        # draw_styled_landmarks(image, results)
    
        while True:
            ret, frame = cap.read()
            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(image, results)
            cv2.putText(image, 'Ready? Press "R" ! :)', (100, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)
            cv2.imshow('frame', image)
            if cv2.waitKey(25) == ord('r'):
                break
        
        # Show to screen
        # cv2.imshow('OpenCV Feed', image)
    
        counter = 0
        while counter < dataset_size:
            ret, frame = cap.read()
            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(image, results)
            cv2.imshow('frame', image)
            cv2.waitKey(25)
            cv2.imwrite(os.path.join(DATA_DIR, str(j), '{}.jpg'.format(counter)), frame)
    
            counter += 1
        
cap.release()
cv2.destroyAllWindows()

C:\Users\kavin\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

In [23]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical 

actions = np.array(['hands_up', 'left_up', 'hands_crossed'])

label_map =   {label:num for num, label in enumerate(actions)} 

sequences, labels = [], []

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for dir_ in os.listdir(DATA_DIR):
        for img_path in os.listdir(os.path.join(DATA_DIR, dir_)):
            img = cv2.imread(os.path.join(DATA_DIR, dir_, img_path))
            # img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            image_land, results = mediapipe_detection(img, holistic)
            draw_styled_landmarks(image_land, results)

            keypoints = extract_keypoints(results)
            sequences.append(keypoints)

            labels.append(dir_)
            
            # plt.figure()
            # plt.imshow(cv2.cvtColor(image_land, cv2.COLOR_BGR2RGB))
    
    # plt.show()

# results.pose_landmarks.landmark

NameError: name 'DATA_DIR' is not defined

In [ ]:
# pose, left_hand, right_hand = [], [], []

# if results.pose_landmarks:
#     for element in results.pose_landmarks.landmark:
#         arr = np.array([element.x, element.y, element.z, element.visibility]).flatten()
#         pose.append(arr)
# else:
#     pose = np.zeros(33*4)

# if results.left_hand_landmarks:
#     for element in results.left_hand_landmarks.landmark:
#         arr = np.array([element.x, element.y, element.z])
#         left_hand.append(arr)
# else:
#     left_hand = np.zeros(21*3)

# if results.right_hand_landmarks:
#     for element in results.right_hand_landmarks.landmark:
#         arr = np.array([element.x, element.y, element.z]).flatten()
#         right_hand.append(arr)
# else:
#     right_hand = np.zeros(21*3)


In [ ]:
# def extract_keypoints(results):
#     pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
#     lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
#     rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
#     return np.concatenate([pose, lh, rh])

In [ ]:
np.array(sequences).shape

In [451]:
np.array(labels).shape

(300,)

In [452]:
x = np.array(sequences)

In [453]:
x.shape

(300, 258)

In [454]:
x_final = x.reshape(300, 1, 258)
x_final.shape

(300, 1, 258)

In [455]:
y = to_categorical(labels).astype(int)

In [456]:
y.shape

(300, 3)

In [457]:
X_train, X_test, y_train, y_test = train_test_split(x_final, y, test_size=0.06)

In [458]:
y_train.shape

(282, 3)

In [459]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [460]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [461]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape = (1, 258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

C:\Users\kavin\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [462]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - categorical_accuracy: 0.3542 - loss: 1.0979
Epoch 2/2000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - categorical_accuracy: 0.4605 - loss: 1.0885 
Epoch 3/2000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - categorical_accuracy: 0.6921 - loss: 1.0306 
Epoch 4/2000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - categorical_accuracy: 0.6893 - loss: 0.6910 
Epoch 5/2000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.7092 - loss: 0.4271 
Epoch 6/2000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 0.9881 - loss: 0.1576 
Epoch 7/2000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - categorical_accuracy: 0.9919 - loss: 0.0419 
Epoch 8/2000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 1.0000 - loss: 0.0084 
Epoch 9/2000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - categorical_accuracy: 0.9955 - loss: 0.0042 
Epoch 10/2000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - categorical_accuracy: 1.0000 - loss: 0.0010     
Epoch 11/2000
9/9 ━━━━━━━━

In [464]:
model.summary()

Model: "sequential_30"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_56 (LSTM)                       │ (None, 1, 64)               │          82,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_57 (LSTM)                       │ (None, 1, 128)              │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_58 (LSTM)                       │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_39 (Dense)                     │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_40 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_41 (Dense)                     │ (None, 3)                   │              99 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 711,755 (2.72 MB)

 Trainable params: 237,251 (926.76 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 474,504 (1.81 MB)

In [465]:
y_predict = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step


In [466]:
y_train.shape

(282, 3)

In [467]:
np.array(y_predict).shape

(18, 3)

In [471]:
actions[np.argmax(y_predict[2])]

'hands_up'

In [472]:
actions[np.argmax(y_test[2])]

'hands_up'

In [473]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [474]:
ytrue = np.argmax(y_test, axis=1).tolist()
y_predict = np.argmax(y_predict, axis=1).tolist()

In [475]:
multilabel_confusion_matrix(ytrue, y_predict)

array([[[12,  0],
        [ 0,  6]],

       [[12,  0],
        [ 0,  6]],

       [[12,  0],
        [ 0,  6]]], dtype=int64)

In [484]:
accuracy_score(ytrue, y_predict)

1.0

In [9]:
import tensorflow as tf 
from tensorflow import keras 

model.save('pose_recognition_model.h5')

NameError: name 'model' is not defined

In [11]:
import tensorflow as tf 
from tensorflow import keras
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import pickle

model = tf.keras.models.load_model('pose_recognition_model.h5')
print("Sucess")

Sucess


In [24]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [25]:
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        keypoints_per_frame = extract_keypoints(results)
        predict_pose = model.predict(keypoints_per_frame.reshape(1,1,258))

        pred_pose_reshape = predict_pose.reshape((3,))
        
        print(actions[np.argmax(pred_pose_reshape)])

        image = prob_viz(pred_pose_reshape, actions, image, colors)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

C:\Users\kavin\AppData\Roaming\Python\Python312\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
hands_crossed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
hands_crossed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
hands_crossed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
hands_crossed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
hands_crossed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
hands_crossed
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
hands_up
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
hands_up
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
hands_up
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
hands_up
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
hands_up
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
hands_up
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
hands_up
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
hands_up
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
hands_up
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
hands_up
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
left_up
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
left_up
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
left_up
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
left_up
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/ste

In [479]:
cv2.putText??

Docstring:
putText(img, text, org, fontFace, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]]) -> img
.   @brief Draws a text string.
.   
.   The function cv::putText renders the specified text string in the image. Symbols that cannot be rendered
.   using the specified font are replaced by question marks. See #getTextSize for a text rendering code
.   example.
.   
.   @param img Image.
.   @param text Text string to be drawn.
.   @param org Bottom-left corner of the text string in the image.
.   @param fontFace Font type, see #HersheyFonts.
.   @param fontScale Font scale factor that is multiplied by the font-specific base size.
.   @param color Text color.
.   @param thickness Thickness of the lines used to draw a text.
.   @param lineType Line type. See #LineTypes
.   @param bottomLeftOrigin When true, the image data origin is at the bottom-left corner. Otherwise,
.   it is at the top-left corner.
Type:      builtin_function_or_method